<a href="https://colab.research.google.com/github/narendraakumar/SDNAWORK/blob/main/Copy_of_pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 16.93 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.


In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... canceled
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resol

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_pix2pix_dataset.sh [cityscapes, night2day, edges2handbags, edges2shoes, facades, maps]`

Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

In [ ]:
!bash ./datasets/download_pix2pix_dataset.sh facades

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
!bash ./scripts/download_pix2pix_model.sh sat2map

# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [ ]:
!python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA --display_id -1 --gpu_ids 0


In [ ]:
# prompt: give perameter gpu id

!python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA --display_id -1 --gpu_ids 1


In [ ]:
# prompt: Traceback (most recent call last):
#   File "/content/pytorch-CycleGAN-and-pix2pix/train.py", line 28, in <module>
#     opt = TrainOptions().parse()   # get training options
#   File "/content/pytorch-CycleGAN-and-pix2pix/options/base_options.py", line 136, in parse
#     torch.cuda.set_device(opt.gpu_ids[0])
#   File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 420, in set_device

!python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA --display_id -1 --gpu_ids 0


# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [ ]:
!ls checkpoints/

In [ ]:
!python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_label2photo_pretrained --use_wandb

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import re
import numpy as np

# Function to concatenate images horizontally (along X-axis)
def concatenate_images(image_list, save_path, base_name):
    # Load images using OpenCV
    images = [cv2.imread(img) for img in image_list]

    # Resize images to the same height (optional)
    if any(img is None for img in images):
        print(f"Error: Failed to load one or more images for {base_name}")
        return  # Skip concatenation if any image is None

    heights = [img.shape[0] for img in images]
    max_height = max(heights)

    resized_images = []
    for img in images:
        scale_ratio = max_height / img.shape[0]
        new_width = int(img.shape[1] * scale_ratio)
        resized_img = cv2.resize(img, (new_width, max_height))
        resized_images.append(resized_img)

    # Concatenate images along X-axis
    concatenated_image = np.hstack(resized_images)

    # Save the concatenated image
    save_file = os.path.join(save_path, base_name + "_concat.jpg")
    cv2.imwrite(save_file, concatenated_image)
    print(f"Saved: {save_file}")

# Function to find images, group by name, and concatenate
def process_images(root_folder, save_folder):
    image_groups = {}

    # Stage 1 folder traversal
    for stage_folder in [os.listdir(root_folder)[1]]:
        stage_path = os.path.join(root_folder, stage_folder)
        if os.path.isdir(stage_path):
            # Go through each image file in the stage folder
            for file in os.listdir(stage_path):
                if file.endswith(('.png', '.jpg', '.jpeg')):  # Adjust image extensions as needed
                    # Extract the base name without the underscore and extension
                    base_name = re.sub(r'_[^_]*$', '', os.path.splitext(file)[0])

                    # Add the image to its group (based on base_name)
                    image_path = os.path.join(stage_path, file)
                    if base_name not in image_groups:
                        image_groups[base_name] = []
                    image_groups[base_name].append(image_path)

    # Make sure the save directory exists
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Concatenate images with the same base name
    for base_name, image_list in image_groups.items():
        if len(image_list) > 1:
            import numpy as np
            image_list = np.sort(image_list)
            print(image_list)

            concatenate_images(image_list, save_folder, base_name)

    # Make sure save directory exists

# Define your folder paths
root_folder = '/content/drive/MyDrive/test'  # Folder where images are stored
save_folder = '/content/data'  # Folder where concatenated images will be saved

# Run the script
process_images(root_folder, save_folder)

['/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000002.jpg'
 '/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000002_.jpg']
Saved: /content/data/PTRC319KWJR1W1F0000002_concat.jpg
['/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000001.jpg'
 '/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000001_.jpg']
Saved: /content/data/PTRC319KWJR1W1F0000001_concat.jpg
['/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000000.jpg'
 '/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000000_.jpg']
Saved: /content/data/PTRC319KWJR1W1F0000000_concat.jpg
['/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000003.jpg'
 '/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000003_.jpg']
Saved: /content/data/PTRC319KWJR1W1F0000003_concat.jpg
['/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000007.jpg'
 '/content/drive/MyDrive/test/PTRC319KWJR1W1F0/PTRC319KWJR1W1F0000007_.jpg']
Saved: /content/data